# Introduction
This notebook shows how to create a policy

In [ ]:
%use roboquant
Welcome()

## Coding our own Order types

We're now ready to develop a "custom policy". For this first example we extend the DefaultPolicy and use our own Order type.

In [ ]:
class MyPolicy(val percentage: Double=0.05) : DefaultPolicy() {

        override fun createOrder(signal: Signal, qty: Double, price: Double): Order? {
            
            // We don't short and all other sell orders are covered by the bracket order.
            if (qty < 0) return null
            
            val asset = signal.asset
      
            return BracketOrder(
                MarketOrder(asset, qty),
                LimitOrder(asset, -qty, price * (1 + percentage)),
                StopOrder(asset, -qty, price* (1 - percentage))
            )
        }
}

## Create a new roboquant and evaluate it
We are now ready to back-test our strategy. And we decided to test it against Apple stock. So what we need to do is:

- Use a data feed that contains historic prices for the Apple stock
- Create an instance of our strategy and tell it to focus on the Apple stock
- Create an instance of a roboquant and run it against the available data 

In [ ]:
val feed = AvroFeed.sp500()
val strategy = EMACrossover()
val policy = MyPolicy()
val roboquant =  Roboquant(strategy, AccountSummary(), policy = policy)

roboquant.run(feed)

## Get some insights
Let's start to see how the account looks like after the expirement has run. The account holds the latest state

In [ ]:
val account = roboquant.broker.account
account.summary()

By default we start a simulated broker account with a default of 1_000_000.00 USD cash. Lets see much realized profit we actually made with out trades:

In [ ]:
account.trades.realizedPnL()

## Lessons learned

* It is not that hard to develop a new strategy and back-test it. 
* Even the most silly strategies can **appear** to make a profit if testing is based on unsound assumptions. In this case running our back test solely against Apple stocks and only going long, we were almost sure to make a profit.  

In [ ]:
account.trades.take(10).summary()

In [ ]:
account.portfolio.summary()